## 2.3 自动求梯度

In [1]:
import tensorflow as tf
tf.enable_eager_execution()
import numpy as np

/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

### 2.3.1 简单例子

我们先看一个简单例子：对函数  y=2x⊤x  求关于列向量  x  的梯度。我们先创建变量x，并赋初值。

In [5]:
x = tf.constant(np.arange(4).reshape((4, 1)))
x = tf.cast(x, 'float32')
x

<tf.Tensor: id=10, shape=(4, 1), dtype=float32, numpy=
array([[0.],
       [1.],
       [2.],
       [3.]], dtype=float32)>

In [9]:
with tf.GradientTape() as tape:
    tape.watch(x)
    y = 2 * tf.transpose(x) @ x
    dy_dx = tape.gradient(target=y, sources=x)

dy_dx

<tf.Tensor: id=68, shape=(4, 1), dtype=float32, numpy=
array([[ 0.],
       [ 4.],
       [ 8.],
       [12.]], dtype=float32)>

### 2.3.3 对Python控制流求梯度

使用Tensorflow的一个便利之处是，即使函数的计算图包含了Python的控制流（如条件和循环控制），我们也有可能对变量求梯度。

In [15]:
def f(a):
    b = a * 2
    while tf.norm(b) < 1000:
        b = b * 2
    if tf.reduce_sum(b) > 0:
        c = b
    else:
        c = 100 * b
    return c


In [31]:
a = tf.random.normal(shape=(1,), dtype='float32')
with tf.GradientTape() as tape:
    tape.watch(a)
    c = f(a)
dc_da = tape.gradient(target=c, sources=a)
dc_da

<tf.Tensor: id=2759, shape=(1,), dtype=float32, numpy=array([409600.], dtype=float32)>

我们来分析一下上面定义的f函数。事实上，给定任意输入a，其输出必然是 f(a) = x * a的形式，其中标量系数x的值取决于输入a。由于c = f(a)有关a的梯度为x，且值为c / a，我们可以像下面这样验证对本例中控制流求梯度的结果的正确性。

In [32]:
tf.equal(dc_da, c/a)

<tf.Tensor: id=2765, shape=(1,), dtype=bool, numpy=array([ True])>